<a href="https://colab.research.google.com/github/zzaeyun/ESAA23_1/blob/main/%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C_487to496.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**04 텍스트 분류 실습 - 20 뉴스그룹 분류**

- 사이킷런의 fetch_20newsgroups() API를 이용해 뉴스그룹의 분류를 수행할 수 있는 예제 데이터 제공
- 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬의 형태
- 이러한 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등
  - 이 중에서 로지스틱 회귀 수행
- 텍스트를 기반으로 분류 수행할 때는 텍스트를 정규화한 뒤 피처 벡터화를 적용

<br>

**텍스트 정규화**



In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data=fetch_20newsgroups(subset='all', random_state=156)

In [4]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- target 클래스 값은 0부터 19까지 20개로 구성됨

<br>

- 개별 데이터가 텍스트로 어떻게 구성돼 있는지 데이터 한 개만 추출

In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있음
- 이중에서 내용을 제외한 정보는 제거
  - 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스 그룹 분류의 target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문
  - 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스 그룹에 속하는지 분류

In [9]:
from sklearn.datasets import fetch_20newsgroups

train_news=fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),
                   random_state=156)
X_train=train_news.data
y_train=train_news.target

test_news=fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),
                   random_state=156)
X_test=test_news.data
y_test=test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


**피처 벡터화 변환과 머신러닝 모델 학습/예측/평가**

- 학습 데이터는 11314개의 뉴스그룹 문서가 리스트 형태로 주어지고, 테스트 데이터는 7532개의 문서가 역시 리스트 형태로 주어짐
- CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화
  - 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야 함
  

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect=CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect=cnt_vect.transform(X_train)

X_test_cnt_vect=cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


- 단어가 101631개로 만들어 졌음

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred=lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.606


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


정확도는 약 0.617

<br>

- 이번에는 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [13]:
 from sklearn.feature_extraction.text import TfidfVectorizer

 tfidf_vect=TfidfVectorizer()
 tfidf_vect.fit(X_train)
 X_train_tfidf_vect=tfidf_vect.transform(X_train)
 X_test_tfidf_vect=tfidf_vect.transform(X_test)

In [14]:
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공

<br>

텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 방법 2가지
- 최적의 ML 알고리즘을 선택
- 최상의 피처 전처리를 수행
  - 텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있음

  <br>

- TF-IDF에 좀 더 다양한 파라미터를 적용

In [15]:
# stop words 필터링 추가
tfidf_vect=TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.692


- GridSearchCV를 이용해 하이퍼 파라미터 최적화 수행
- 로지스틱 회귀의 C 파라미터만 변경하면서 최적의 C값을 찾은 뒤 이 C 값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [16]:
from sklearn.model_selection import GridSearchCV

params={'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr=GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Logistic Regression best C parameter: {'C': 10}


In [17]:
pred=grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


C가 10일 때 GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타내었고, 약간 향상된 성능 수치가 되었음

**사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합**
- 사이킷런의 Pipeline 클래스를 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행 가능
- 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있음




```
pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
                   ('lr_clf', LogisticRegression(random_state=156))])
```

- TfidfVectorizer 객체를 tfidf_vect라는 객체 변수 명으로, LogistciRegression 객체를 lr_clf라는 객체변수 명으로 생성한 뒤 이 두 개의 객체를 파이프 라인으로 연결하는 pipeline을 생성한다는 의미
- TfidfVectorizer와 LogisticRegressor의 fit과 predict는 통일되어 수행됨



In [18]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
                   ('lr_clf', LogisticRegression(C=10))])

In [19]:
pipeline.fit(X_train, y_train)
pred=pipeline.predict(X_test)

print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


다음 예제는 GridSearchCV에 Pipeline을 입력하면서 TfidfVectorizer의 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화
- 이때 Pipeline을 입력할 경우에는 param_grid의 입력 값을 딕셔너리 형태의 Key와 Value 값을 가지는데, **하이퍼 파라미터 명이 객체 변수명과 결합되어 제공** ex) tfidf_vect__ngram_range
- 유의할 점은 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점

In [ ]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
                   ('lr_clf', LogisticRegression())])

params={'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
        'tfidf_vect__max_df': [100, 300, 700],
        'lr_clf__C':[1,5,10]}

grid_cv_pipe=GridSearchCV(pipeline,param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)

print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)

pred=grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))